**Pakistan Largest Ecommerce Dataset EDA, Time Series Forecasting, Classification**

# Table of Contents

- 1. Install Required Libraries
- 2. Import Libraries
- 3. Load Data
- 4. Exploratory Data Analysis (EDA)
   - 4.1 Head of Dataset
   - 4.2 Schema of data
   - 4.3 Shape of data
   - 4.4 Summary statistics of data
   - 4.5 Price Distribution
- 5. Fix Data Quality Problems
   - 5.1 Find missing values from data
   - 5.2 Remove null columns
   - 5.3 Drop rows where all values are null
   - 5.4 Fill null values
   - 5.5 Removing null rows on the basis of Working Date and sku column
   - 5.6 Fix the Data Types of Columns by performaing type casting
   - 5.7 Remove data duplications if any exist in data
   - 5.8 Shape of data after preprocessing
   - 5.9 Data types of data after type casting
   - 5.10 Summary Statistics of processed data
- 6. Exploratory Data Analysis (EDA)
   - 6.1 Correlation Between Columns
   - 6.2 Best Selling category
   - 6.3 Visulize Payment Methods versus Order Status
   - 6.4 Order Status per Year
   - 6.5 Payment method vs Orders
   - 6.6 No. of Orders per Month of each year
   - 6.7 Best Category by Completion order
   - 6.8 Worst Category by not Completion order
- 7. Time Series Forecasting of Daily Sales
   - 7.1 Extracting feature day wise order count
   - 7.2 Order count graph w.r.t date
   - 7.3 Facebook Prophet Model for Daily Order Count Forecasting
   - 7.4 Performance Evaluation
- 8. ML Model for Classification of Order Status
   - 8.1 Graphical Confusion Matrix Code
   - 8.2 Data Prepration for Model Input
   - 8.3 Converting Data to Features
   - 8.4 Split data to Training and Testing
   - 8.5 Random Forest Classifier
       - 8.5.1 Predictions
       - 8.5.2 Performance Evaluation
   - 8.6 Logistic Regression Classifier
       - 8.5.1 Predictions
       - 8.6.2 Performance Evaluation

## 1. Install Required Libraries

In [ ]:
# install pyspark
!pip install pyspark

## 2. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import itertools

from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import add_changepoints_to_plot, plot_cross_validation_metric

from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import classification_report, confusion_matrix

# Import Sparksession
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

import warnings
warnings.filterwarnings('ignore')

spark=SparkSession.builder.appName("Data_Wrangling").getOrCreate()

In [ ]:
# Print PySpark and Python versions
print('Python version: '+sys.version)
print('Spark version: '+spark.version)

## 3. Load Data

In [ ]:
# Read data
file_location = "../input/pakistans-largest-ecommerce-dataset/Pakistan Largest Ecommerce Dataset.csv"
file_type = "csv"
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type)\
.option("inferSchema", infer_schema)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load(file_location)

## 4. Exploratory Data Analysis(EDA)

### 4.1 Head of dataset

In [ ]:
df.show()

### 4.2 Schema of data

In [ ]:
# Print Metadata
df.printSchema()

### 4.3 Shape of data

In [ ]:
print('The shape of ecommerce dataset is', (df.count(), len(df.columns)))

### 4.4 Summary statistics of data

Summary statistic of data and to easily see all the columns clearly without scroll I take the Transpose of statistics result.

In [ ]:
df.describe().toPandas().T

### 4.5 Price Distribution

In [ ]:
plt.figure(figsize=(12, 6))
sns.distplot([row['price'] for row in df.select('price').collect()], color='g');

Note: For further Exploratory Data Analysis I need to fix the problems in data after fixing the data I can coninue the Exploratory Data Analysis

## 5. Fix Data Qulaity Problems

### 5.1 Find missing values from data

In [ ]:
# Identifying missing values
df.select([count(when((col(c)=='') | col(c).isNull() |isnan(c), c)).alias(c) for c in df.columns]).show()

### 5.2 Remove null columns

In [ ]:
#  Drop Columns
drop_columns_list=["_c21","_c22","_c23","_c24","_c25"]
df1 = df.drop(*drop_columns_list)

In [ ]:
df1.show()

Data contains 1048574 rows but 464051 rows have no records. 584524 rows contain records

By analysing data I observe that almost half of the rows are completely empty. I also observe that there are five column which are totally empty so i removed that columns from data. By further analysing I observe that i can't drop alll Null values rows because some actual data columns have null values so I need to keep them. I will only drop those rows where all values are null.

only left those column where are records

In [ ]:
df1.columns

### 5.3 Drop rows where all values are null

Now I will remove where all rows are empty. If we see the tail of data we will see few of rows where all columns are empty.

In [ ]:
df1.orderBy(asc("item_id")).show(5)

In [ ]:
df1 = df1.na.drop(how = "all")

### 5.4 Fill null values

fill some null rows with mode value of a column.

In [ ]:
# Identifying missing values
df1.select([count(when((col(c)=='') | col(c).isNull() |isnan(c), c)).alias(c) for c in df1.columns]).show()

Filling missing values of status and category_name_1 column by mode of column values

In [ ]:
## computing mode value of status column for fill missing value
mode_status = df1.groupby("status").count().orderBy("count", ascending=False).first()[0]
mode_status

In [ ]:
## mode value of status column is complete 
df1 = df1.fillna(mode_status, subset=['status'])

In [ ]:
## computing mode value of category_name_1 column for fill missing value
mode_category_name_1 = df1.groupby("category_name_1").count().orderBy("count", ascending=False).first()[0]
mode_category_name_1

In [ ]:
df1 = df1.fillna(mode_category_name_1, subset=['category_name_1'])

check null values

In [ ]:
# Identifying missing values
df1.select([count(when((col(c)=='') | col(c).isNull() |isnan(c), c)).alias(c) for c in df1.columns]).show()

### 5.5 Removes null rows on the basis of Working Date and sku column

In [ ]:
df1 = df1.na.drop(subset=['Working Date', 'sku', 'Customer ID'])

In [ ]:
# Identifying missing values
df1.select([count(when((col(c)=='') | col(c).isNull() |isnan(c), c)).alias(c) for c in df1.columns]).show()

### 5.6 Fix the Data Types of Columns by performaing type casting

In [ ]:
df1 = df1.withColumn('created_at', F.to_date(F.unix_timestamp('created_at', 'M/d/y').cast('timestamp')))
df1 = df1.withColumn('Working Date', F.to_date(F.unix_timestamp('Working Date', 'M/d/y').cast('timestamp')))

In [ ]:
df1.printSchema()

In [ ]:
df1.show()

In [ ]:
df1 = df1.withColumn("qty_ordered", df1["qty_ordered"].cast(IntegerType()))
df1 = df1.withColumn("price", df1["price"].cast(IntegerType()))
df1 = df1.withColumn("grand_total", df1["grand_total"].cast(IntegerType()))
df1 = df1.withColumn("discount_amount", df1["discount_amount"].cast(IntegerType()))
df1 = df1.withColumn("Month", df1["Month"].cast(IntegerType()))
df1 = df1.withColumn("Year", df1["Year"].cast(IntegerType()))

Remove extra space from column name MV and rename column created_at to order_date

In [ ]:
df1 = df1.withColumnRenamed(" MV ", "MV")
df1 = df1.withColumnRenamed("created_at", "order_date")

### 5.7 Remove data duplications if any exist in data

In [ ]:
df1 = df1.drop_duplicates()

### 5.8 Shape of data after preprocessing

In [ ]:
print('The shape of ecommerce dataset is', (df1.count(), len(df1.columns)))

### 5.9 Data types of data after type casting

In [ ]:
df1.printSchema()

### 5.10 Summary Statistics of processed data

In [ ]:
df1.describe().toPandas().T

## 6. Exploratory Data Analysis(EDA) Continue's

### 6.1 Correlation Between Columns

In [ ]:
from pyspark.ml.stat import Correlation
data = df1.select('price', 'qty_ordered', 'grand_total', 'discount_amount', 'Year', 'Month')
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=data.columns, 
                            outputCol=vector_col)
myGraph_vector = assembler.transform(data).select(vector_col)
matrix = Correlation.corr(myGraph_vector, vector_col).collect()[0][0]

In [ ]:
matrix = Correlation.corr(myGraph_vector, vector_col).collect()[0][0]
corrmatrix = matrix.toArray().tolist()
print(corrmatrix)

In [ ]:
correlation_dataframe = spark.createDataFrame(corrmatrix, data.columns)
correlation_dataframe.show()

In [ ]:
plt.figure(figsize = (12,10))
ax = sns.heatmap(correlation_dataframe.toPandas(), annot=True)
ax.set_yticklabels(correlation_dataframe.columns)
plt.show()

### 6.2 Best Selling category


In [ ]:
## all categories
df1.select("category_name_1").distinct().show()

In [ ]:
best_category = df1.groupby("category_name_1").count().sort(col("count").desc()).toPandas()
best_category

In [ ]:
best_category.set_index('category_name_1', inplace=True)
ax = best_category.plot(kind='pie', y='count', autopct='%1.1f%%', figsize=(8,8), title="Best Selling Categories")
ax.legend(bbox_to_anchor=(1.5, 1.0))
plt.show()

In [ ]:
best_category.plot(kind='bar', y='count', figsize=(12,6),\
                   title="Best Selling Categories")
plt.ylabel("Total Order")
plt.show()

Mobiles & Tablets are Best Selling category in Ecommerce.  
70% of the items sell from these seven categories , Mobiles & Tablets  ,  Men's Fashion ,  Women's Fashion , Appliances  , Superstore  , Beauty & Grooming  , soghaat  contributed.

### 6.3 Visualize Payment Methods versus Order Status

In [ ]:
status_count = df1.groupby("status").count().sort(col("count").desc()).toPandas()
status_count

In [ ]:
status_count.set_index('status', inplace=True)
status_count.plot(kind='bar', y='count', figsize=(12,6),\
                  title="Order Status Frequency", color='red')
plt.xlabel("status category")
plt.ylabel("total counts")
plt.show()

### 6.4 Order Status per Year

In [ ]:
df1.select("Year").distinct().sort(col("Year")).show()

In [ ]:
n = 1
plt.figure(figsize=(22 , 10))
for year in df1.select("Year").distinct().sort(col("Year")).collect():
  data = [val['status'] for val in df.filter(F.col("Year") == year['Year'])\
          .select('status').collect() if val['status'] != None]
  plt.subplot(1,3,n)
  plt.title('Order Status in {0}'.format(year['Year']))
  plt.xlabel('Order status')
  plt.ylabel('Total no of orders')
  plt.xticks(rotation = 90)
  plt.hist(data)
  n+=1

plt.show()

I observe from the graphs in years 2016, 2017, 2017 almost completed order quantities are as follows 74000, 123400, 35600. But if we see the canclecd orders in years 2016, 2017, 2017 are as follows almost 40000, 95000, 67000. Also the refunded order of years 2016, 2017, 2017 as follow almost 14000, 3400, 1300.   

I observe that order completion is decreasing and order canceletion is increasing which is not good sign for Ecommerece. Ecommerece website should have to make a policy which help them to maintain thier customer and order completion is successful.  

### 6.5 Payment method vs Orders

In [ ]:
df1.groupby("payment_method").count().show(20, False)

Cod replace with his full name cash_on_Delivery

In [ ]:
df1 = df1.withColumn('payment_method', regexp_replace('payment_method', 'cod', 'cash_on_delivery'))

In [ ]:
payment_method_count = df1.groupby('payment_method').count().sort(col("count").desc()).toPandas()
payment_method_count

In [ ]:
payment_method_count.set_index('payment_method', inplace=True)
payment_method_count.plot(kind='bar', figsize=(12,6) , color='green' , title="Payment method vs order")
plt.xlabel("payment category")
plt.ylabel("total counts")
plt.show()

### 6.6 No. of Orders per Month of each Year

In [ ]:
order_per_month_year = df1.groupby('M-Y').count().sort(col("count").desc()).toPandas()
order_per_month_year

In [ ]:
order_per_month_year.set_index('M-Y', inplace=True)
order_per_month_year.plot(kind='bar', figsize=(12,6) , color='Blue' , title="Order count per month of each year")
plt.xlabel("Month-Year")
plt.ylabel("total counts")
plt.show()

### 6.7 Best Category by Completion order

In [ ]:
oreder_completed = df1.filter(F.col('status').isin(['complete','paid', 'received', 'cash_on_delivery']))
oreder_completed_by_category = oreder_completed.groupby('category_name_1').count()\
.sort(col("count").desc()).toPandas()

oreder_completed_by_category

In [ ]:
oreder_completed_by_category.set_index('category_name_1', inplace=True)
oreder_completed_by_category.plot(kind='bar', figsize=(12,6), color='purple',
                                  title="Best Cateory w.r.t Order Completion")
plt.xlabel("Category")
plt.ylabel("total counts")
plt.show()

### 6.8 Worst Category by not Completion order

In [ ]:
oreder_not_completed = df1.filter(~F.col('status').isin(['complete','paid', 'received', 'cash_on_delivery']))
oreder_not_completed_by_category = oreder_not_completed.groupby('category_name_1').count()\
.sort(col("count").desc()).toPandas()

oreder_not_completed_by_category

In [ ]:
oreder_not_completed_by_category.set_index('category_name_1', inplace=True)
oreder_not_completed_by_category.plot(kind='bar', figsize=(12,6), color='red',
                                  title="Worst Cateory w.r.t not Order Completion")
plt.xlabel("Category")
plt.ylabel("total counts")
plt.show()

## 7. Time Series Forecasting of Daily Sales

### 7.1 Extracting feature day wise order count

In [ ]:
data = df1.groupby("order_date").count().sort(col("order_date")).toPandas()
data.rename(columns={ "order_date": 'ds', "count":"y"}, inplace=True)
data.head()

### 7.2 Order count graph w.r.t date

In [ ]:
plot_data = data.copy()
plot_data.index = plot_data.ds
plot_data.drop(['ds'], axis=1, inplace=True)
plot_data.plot(kind='line', figsize=(16, 10), title='Order Count w.r.t Date')

### 7.3 Facebook Prophet Model for Daily Order Count Forecasting

In [ ]:
model = Prophet(daily_seasonality=True)
model.fit(data)

In [ ]:
 # predicting out of sample for next year
future = model.make_future_dataframe(periods=365, freq='d', include_history=True)
future.tail(4)

In [ ]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
fig1 = model.plot(forecast,xlabel="Date",ylabel="Number of Orders")

### 7.4 Performance Evaluation

In [ ]:
# Execute cross validation
df_cv = cross_validation(model, initial='365 days', period='90 days', horizon = '60 days')
pm = performance_metrics(df_cv)
display(pm.head(),pm.tail())
fig = plot_cross_validation_metric(df_cv, metric='mape')
plt.show()

In [ ]:
df_cv

## Forecasting of order count by category

In [ ]:
df.select('category_name_1').distinct().show()

## Forecasting for Mobiles & Tablets category

In [ ]:
mbl_cat_data = df1.filter(col("category_name_1") == 'Mobiles & Tablets')

In [ ]:
data1 = mbl_cat_data.groupby("order_date").count().sort(col("order_date")).toPandas()
data1.rename(columns={ "order_date": 'ds', "count":"y"}, inplace=True)
data1.head()

In [ ]:
model_1 = Prophet(daily_seasonality=True)
model_1.fit(data1)

In [ ]:
 # predicting out of sample for next year
future = model_1.make_future_dataframe(periods=60, freq='d', include_history=True)
future.tail(4)

In [ ]:
forecast = model_1.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
fig1 = model_1.plot(forecast, xlabel="Date",ylabel="Number of Orders")

In [ ]:
# Execute cross validation
df_cv = cross_validation(model_1, initial='500 days', period='60 days', horizon = '60 days')
pm = performance_metrics(df_cv)
display(pm.head(),pm.tail())
fig = plot_cross_validation_metric(df_cv, metric='mape')
plt.show()

In [ ]:
df_cv

## Forecasting for Health & Sports category

In [ ]:
h_s_data = df1.filter(col("category_name_1") == 'Health & Sports')

In [ ]:
data2 = h_s_data.groupby("order_date").count().sort(col("order_date")).toPandas()
data2.rename(columns={ "order_date": 'ds', "count":"y"}, inplace=True)
data2.head()

In [ ]:
model_2 = Prophet()
model_2.fit(data2)

In [ ]:
 # predicting out of sample for next year
future = model_2.make_future_dataframe(periods=60, freq='d', include_history=True)
future.tail(4)

In [ ]:
forecast = model_2.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
fig1 = model_2.plot(forecast, xlabel="Date",ylabel="Number of Orders")

In [ ]:
# Execute cross validation
df_cv = cross_validation(model_2, initial='500 days', period='60 days', horizon = '60 days')
pm = performance_metrics(df_cv)
display(pm.head(),pm.tail())
fig = plot_cross_validation_metric(df_cv, metric='mape')
plt.show()

In [ ]:
df_cv

## 8. ML Model for Classification of Order Status

### 8.1 Graphical Confusion Matrix Code

In [ ]:
# Graphical Confusion Matrix function
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### 8.2 Data Prepration for Model Input

In [ ]:
categorical_columns= ['category_name_1', 'payment_method', 'BI Status', 'Customer Since']

stage_strings= [StringIndexer(inputCol= c, outputCol= c+"_string_encoded") for c in categorical_columns]
stage_string = [StringIndexer(inputCol= 'status', outputCol= "status_string_encoded")]
stage_one_hot = [OneHotEncoder(dropLast=True, inputCol= c+"_string_encoded", outputCol= c+ "_one_hot") for c in categorical_columns]

ppl = Pipeline(stages= stage_strings + stage_string + stage_one_hot)
categorical_data = df1.select(*categorical_columns)
df = ppl.fit(df1).transform(df1)

In [ ]:
df = df.select([c for c in df.columns if c not in ['item_id', 'status',
                                                   'order_date', 'sku',
                                                   'price', 'increment_id',
                                                   'sales_commission_code', 
                                                   'Working Date', 'MV', 
                                                   'Year', 'M-Y', 'FY',
                                                   'Customer ID', 'status']+categorical_columns])

In [ ]:
df.show(5)

### 8.3 Converting Data to Features

In [ ]:
vector_assembler = VectorAssembler(inputCols=df.columns, outputCol= "features")
data_training_and_test = vector_assembler.transform(df)

### 8.4 Split data to Training and Testing

In [ ]:
# 70% data for training and 30% data for testing 
(training_data, test_data) = data_training_and_test.randomSplit([0.7, 0.3], 2021)

### 8.5 Random Forest Classifier

In [ ]:
rf = RandomForestClassifier(labelCol="status_string_encoded",
                            featuresCol="features", numTrees=20, seed=2021)
model = rf.fit(training_data)

#### 8.5.1 Predictions

In [ ]:
predictions = model.transform(test_data)

predictions.select("status_string_encoded","prediction", "probability")\
.show(n = 20, truncate = 30)

#### 8.5.2 Performance Evaluation

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='status_string_encoded',
                                              predictionCol="prediction",
                                              metricName= 'accuracy')
f1= MulticlassClassificationEvaluator(labelCol='status_string_encoded',
                                      predictionCol= 'prediction',
                                      metricName= 'f1')
print('Accuracy', evaluator.evaluate(predictions))
print('F1 Score', f1.evaluate(predictions))

In [ ]:
y_true = predictions.select(['status_string_encoded']).collect()
y_pred = predictions.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

In [ ]:
cnf_matrix = confusion_matrix(y_true, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(14, 8))
plot_confusion_matrix(cnf_matrix, classes=range(0, 16)
                      ,normalize= False,  title='Confusion matrix')

### 8.6 Logistic Regression Classifier

In [ ]:
lr = LogisticRegression(labelCol='status_string_encoded',
                        featuresCol='features', predictionCol= 'prediction')
lrModel = lr.fit(training_data)

#### 8.6.1 Predictions

In [ ]:
predictions = lrModel.transform(test_data)

predictions.select("status_string_encoded", "prediction", "probability")\
.show(n=20, truncate=30)

#### 8.6.2 Performance Evaluation

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='status_string_encoded',
                                              predictionCol="prediction",
                                              metricName= 'accuracy')
f1= MulticlassClassificationEvaluator(labelCol='status_string_encoded',
                                      predictionCol= 'prediction',
                                      metricName= 'f1')
print('Accuracy', evaluator.evaluate(predictions))
print('F1 Score', f1.evaluate(predictions))

In [ ]:
y_true = predictions.select(['status_string_encoded']).collect()
y_pred = predictions.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

In [ ]:
cnf_matrix = confusion_matrix(y_true, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(14, 8))
plot_confusion_matrix(cnf_matrix, classes=range(0, 16)
                      ,normalize= False,  title='Confusion matrix')